In [2]:
import pandas as pd
import os
import pickle
from os.path import expanduser
import numpy as np
from scipy.stats import wasserstein_distance as wd

from data_converge.util import data_utils as du


%load_ext autoreload
%reload_ext autoreload
%autoreload 2


# Load prediction data
experiment_path = '/work/projects/SD2E-Community/prod/projects/sd2e-project-43/test/dc_YeastSTATES-CRISPR-Short-Duration-Time-Series-35C_20200423195648'
experiment_df = du.get_data_and_metadata(experiment_path)

/work/07431/rmahtab/jupyter_packages/lib/python3.6/site-packages/FlowCytometryTools-0.5.0-py3.6.egg/FlowCytometryTools/core/docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)
/work/07431/rmahtab/pysd2cat/test-harness/harness/test_harness_class.py:23: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


['/work/projects/SD2E-Community/prod/projects/sd2e-project-43/test/dc_CEN-PK-Inducible-CRISPR-4-Day-Obstacle-Course_20200506021137',
 '/work/projects/SD2E-Community/prod/projects/sd2e-project-43/test/dc_CEN-PK-Inducible-CRISPR-Characterization_20200426200027',
 '/work/projects/SD2E-Community/prod/projects/sd2e-project-43/test/dc_YeastSTATES-Beta-Estradiol-OR-Gate-Plant-TF-Dose-Response_20200414091814',
 '/work/projects/SD2E-Community/prod/projects/sd2e-project-43/test/dc_YeastSTATES-Beta-Estradiol-OR-Gate-Plant-TF-Dose-Response_20200423200308',
 '/work/projects/SD2E-Community/prod/projects/sd2e-project-43/test/dc_YeastSTATES-Beta-Estradiol-OR-Gate-Plant-TF-Growth-Curves_20200414150010',
 '/work/projects/SD2E-Community/prod/projects/sd2e-project-43/test/dc_YeastSTATES-Beta-Estradiol-OR-Gate-Plant-TF-Growth-Curves_20200423201016',
 '/work/projects/SD2E-Community/prod/projects/sd2e-project-43/test/dc_YeastSTATES-CRISPR-Dose-Response_20200414151033',
 '/work/projects/SD2E-Community/prod/pr

In [45]:
# Create renaming dict for channel columns
channels = ['FSC-A', 'SSC-A', 'BL1-A', 'FSC-W', 'FSC-H', 'SSC-W', 'SSC-H']
channels_under = [x.replace('-', '_') for x in channels]
renaming = dict(zip(channels_under, channels))

# Group prediction data
exp_grouped = experiment_df.groupby(['strain_name', 'timepoint', 'inducer_concentration', 'replicate'])

# Set path to control data
XPLAN_PROJECT="sd2e-project-14"
xplan_base = os.path.join(expanduser("~"), 'sd2e-projects', XPLAN_PROJECT)
exp_id = 'r1dag7xpmgar9k_r1dakfqtk7hpzb.csv'
path = os.path.join(xplan_base, 'xplan-reactor', 'data', 'transcriptic')

# Initialize distances dictionary and counter
nearest_control_distances = dict()
count = 0

# Loop through control data files
for exp in os.listdir(path):
    if ".csv" in exp:
        # Print counter
        count += 1
        print(str(count) + "/109 (" + exp + ")")
        
        # Extract current control data and rename channel columns
        exp_flow = pd.read_csv(os.path.join(path, exp), index_col=0)
        controls = exp_flow.loc[(exp_flow.strain_name == "WT-Live-Control") | (exp_flow.strain_name == "WT-Dead-Control")]
        controls = controls.rename(columns=renaming)
        
        # Separate live/dead control data
        dead_controls = controls.loc[controls['strain_name'] == "WT-Dead-Control"]
        live_controls = controls.loc[controls['strain_name'] == "WT-Live-Control"]
        
        # Apply log-10 transform to channel columns
        dc_d = dead_controls[channels].apply(np.log10).replace([np.inf, -np.inf], 0.0)
        lc_d = live_controls[channels].apply(np.log10).replace([np.inf, -np.inf], 0.0)
        
        # Total Wasserstein distance of current control will be sum of distances of each sample to nearest control
        wass_dist = 0
        
        # Loop through samples
        for sample_name, sample_data in exp_grouped:
            # Apply log-10 transform to channel columns
            s_d = sample_data[channels].apply(np.log10).replace([np.inf, -np.inf], 0.0)
            
            # Get Wasserstein distance between each channel of sample to both live/dead controls
            dw = pd.Series()
            lw = pd.Series()
            for channel in channels:
                dw[channel] = wd(dc_d[channel].dropna(), s_d[channel].dropna())
                lw[channel] = wd(lc_d[channel].dropna(), s_d[channel].dropna())
            
            # Average over channels and add nearest control to overall distance
            if dw.mean() < lw.mean():
                wass_dist += dw.mean()
            else:
                wass_dist += lw.mean()
        
        nearest_control_distances[exp] = wass_dist

1/109 (r1dgppg46aeknn_r1dgwpywx6rtc8.csv)


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/work/07431/rmahtab/jupyter_packages/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


2/109 (r1c7cpt3djxuj_r1c7fex29q6t2.csv)
3/109 (r1c84xwx57y95_r1c8ejz8jfg9z.csv)
4/109 (r1cft9hhnhjdb_r1cfwgjmz247y.csv)
5/109 (r1cyeyjwbefaf2_r1cys7qtr6j42x.csv)
6/109 (r1c84y2j7n7bu_r1c8aw59wfxy8.csv)
7/109 (r1c7cpvfzqprk_r1c7fbvba55db.csv)
8/109 (r1cdcetsvxgc2_r1cdfa6jrtssc.csv)
9/109 (r1c8yyg9gxbme_r1c96wn43pq45.csv)
10/109 (r1dj8mxn3pcuf5_r1djfnchgugknq.csv)
11/109 (r1ce3pjymvm3u_r1ceb3rgpe8gm.csv)
12/109 (r1dgppdqbd2mmk_r1dgwy63p8x5uz.csv)
13/109 (r1ce3pqt45vfe_r1ce7qf5e483u.csv)
14/109 (r1ce3ppkyswxf_r1ce7qk6h482m.csv)
15/109 (r1dfxeycahns9y_r1dg8u3z9yzuuq.csv)
16/109 (r1ceuhvgf7b4y_r1cf2yr353x5j.csv)
17/109 (r1cwvvxz5d7wua_r1cx7ucsnkjfsb.csv)
18/109 (r1c84y3r9yqxb_r1c8auwvzbgt3.csv)
19/109 (r1c8yydkumrkr_r1c96xsxw79c9.csv)
20/109 (r1cgbw3xk7pau_r1cgpa54x8ff3.csv)
21/109 (r1cckyh56hjqj_r1ccpyjef2vca.csv)
22/109 (r1cfkpp2p3kjz_r1cftpr8wxqc7.csv)
23/109 (r1c8xx7we38vw_r1c93cukfcxqp.csv)
24/109 (r1cbut53kpf3n_r1cc3bn2s2aj8.csv)
25/109 (r1cdcesknarkf_r1cdfg4x5uuwp.csv)
26/109 (r1dfxe

In [ ]:
nearest_control_distances

In [48]:
# Find experiment with minimum distance
min(nearest_control_distances.values())

77.96594207091962

In [ ]:
# Pickle nearest control Wasserstein distances
nearest_control_distances.to_pickle('wasserstein_distances.pkl')

In [14]:
# OLD Wasserstein distance computation between centroids of controls and prediction data

exp_names = []
for exp in os.listdir(path):
    if ".csv" in exp:
        exp_names.append(exp)

wd_avgs = []
num_channels = len(channels)

for i, row in wass_distances.iterrows():
    channel_sum = 0
    for channel in channels:
        channel_sum += row[channel]
    wd_avgs.append(channel_sum/num_channels)

wass_distances['Average'] = wd_avgs
wass_distances['Experiment Name'] = exp_names

pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Find experiment with minimum distance
wass_distances[wass_distances['Average'] == min(wass_distances['Average'])]